In [55]:
import streamlit as st
import gpxpy
import folium
from streamlit_folium import st_folium
import pandas as pd
import boto3
import json
import random



def parse_gpx(gpxdf, number):
    points = []
    print(gpxdf)
    gpx_data = gpx = gpxpy.parse(gpxdf['activity_gpx']) 
    color = gpxdf['color']
    filename = gpxdf['filename']
    activity_name = gpxdf['activity_name']
    # print(color)
    for track in gpx_data.tracks:
        for segment in track.segments:
            for point in segment.points:
                points.append({'Latitude': point.latitude, 'Longitude': point.longitude, 'Color': color, 'Number' : number, 'activity_name': activity_name, 'filename': filename})
    return points

def bucket_query_namefiles():
    activity_filename = []
    session = boto3.Session()
    s3 = session.resource('s3') 
    bucket = s3.Bucket('solvesdgs')
    for obj in bucket.objects.all():
        if 'activityfiles' in obj.key:
            activity_filename.append(obj.key)
    return activity_filename[1:]

def generate_random_color():
    # Generate a random hexadecimal color code
    color_code = "#{:06x}".format(random.randint(0, 0xFFFFFF))
    return color_code

def download_files():
    session = boto3.Session()
    s3 = session.resource('s3') 
    files_name = bucket_query_namefiles()
    json_df = pd.DataFrame()
    color_random = generate_random_color()
    for obj_name in files_name:
        content_object = s3.Object('solvesdgs', obj_name)
        file_content = content_object.get()['Body'].read()    
        json_content = json.loads(file_content)
        json_content['filename'] = obj_name
        json_content['color'] = color_random
        content_df = pd.DataFrame([json_content],  index=[0])
        content_df= content_df[['activity_gpx','filename', 'color' , 'activity_name']]
        json_df = pd.concat([json_df,content_df], ignore_index=True)
    return json_df

def read_files():
    df_content = download_files()
    # Parse each GPX file
    all_points = []    # for file, color in colors.items():
    for index, row in df_content.iterrows():
        points = parse_gpx(row, index)
        all_points.extend(points)

    return all_points

def plot_map(points):
    m = folium.Map(location=[points[0]['Latitude'], points[0]['Longitude']], zoom_start=12)
    seen = {}
    
    for point in points:
        if not seen.get(point['Number'], False):
            folium.Marker(location=[point['Latitude'], point['Longitude']],  popup= point['activity_name'],
                          icon=folium.Icon(color=point['Color'])).add_to(m)
        seen[point['Number']] = True

    grouped = pd.DataFrame(points).groupby('Number')

    for number_id, group_data in grouped:
        color_1 = group_data['Color'].iloc[0]
        locations = [(row['Latitude'], row['Longitude']) for index, row in group_data.iterrows()]
        folium.PolyLine(locations=locations, color=color_1).add_to(m)
        
    return m

In [56]:
all_points = read_files()

activity_gpx     <?xml version="1.0" encoding="UTF-8"?>\n<gpx x...
filename         activityfiles/128fe47b961aaa66ee39de43b0aed636...
color                                                      #2b3897
activity_name                                            cool hike
Name: 0, dtype: object
activity_gpx     <?xml version="1.0" encoding="UTF-8"?>\n<gpx x...
filename         activityfiles/54ef005ead386218b081b4a48ec2e6a2...
color                                                      #2b3897
activity_name                                        otaranta hike
Name: 1, dtype: object
activity_gpx     <?xml version="1.0" encoding="UTF-8"?>\n<gpx x...
filename         activityfiles/bdae5aa40e89e9e4a19955a0d6d78f9d...
color                                                      #2b3897
activity_name                                             jmt hike
Name: 2, dtype: object
activity_gpx     <?xml version="1.0" encoding="UTF-8"?>\n<gpx x...
filename         activityfiles/e0bfd7885866154f629dee224b364

In [57]:
plot_map(all_points)

/tmp/ipykernel_61955/3440301552.py:75: UserWarning: color argument of Icon should be one of: {'darkgreen', 'lightgreen', 'red', 'gray', 'purple', 'orange', 'cadetblue', 'white', 'lightblue', 'green', 'blue', 'darkred', 'darkblue', 'pink', 'darkpurple', 'lightgray', 'black', 'beige', 'lightred'}.
  icon=folium.Icon(color=point['Color'])).add_to(m)
